In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:

ruta_carpeta = 'C:/Users/Sistemas/Desktop/pallets_cruzados_nuevo/cruce_2025'

# Lista para almacenar DataFrames de cada mes
dataframes_meses = []

# Iterar sobre los archivos en la carpeta
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('_limpio.xlsx') and not archivo.startswith('despacho'):
        mes = archivo.split('_')[0]  # Extraer 'enero', 'febrero', etc.
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        df_mes = pd.read_excel(ruta_archivo)
        df_mes['mes'] = mes  # Opcional: agregar columna con el mes
        dataframes_meses.append(df_mes)

# Combinar todos los DataFrames mensuales en uno solo
df_anual = pd.concat(dataframes_meses, ignore_index=True)

In [3]:
# 3. Cargar despacho y renombrar columnas
df_despachos = pd.read_excel(os.path.join(ruta_carpeta, 'despacho_limpio.xlsx'))
df_despachos = df_despachos.rename(columns={
    'fecha_despacho': 'fecha',
    'cantidad_pallets_sacados': 'cantidad_pallets_sacados'
})

In [4]:
df_anual['fecha'] = pd.to_datetime(df_anual['fecha'], errors='coerce')
df_despachos['fecha'] = pd.to_datetime(df_despachos['fecha'], errors='coerce')

In [5]:

# 5. Hacer el merge LEFT: mantengo todos los datos de pallets y sumo el despacho
df_completo = pd.merge(
    df_anual,
    df_despachos,
    on=['id_socio', 'fecha'],
    how='left'
)

In [6]:
# 6. Rellenar NaN en 'cantidad_pallets_sacados' con 0 si no hubo despacho ese día
df_completo['cantidad_pallets_sacados'] = df_completo['cantidad_pallets_sacados'].fillna(0)

In [7]:
# 7. Verificar
print(df_completo.head())

   id_socio                             socio      fecha  pallets  base  \
0       3.0                        DOZ CARLOS 2025-04-01       14  10.0   
1       5.0                    GARMENDIA HUGO 2025-04-01       26  10.0   
2       8.0                  GRAZIOSI MIRTHA  2025-04-01       32  10.0   
3       9.0  PACI ANTONIO - DIGIANO JAVIER SH 2025-04-01        7  10.0   
4      10.0                    ROSICA GABRIEL 2025-04-01        4  10.0   

   Promedio pallets por socio diario Porcentaje pallets por socio  \
0                               15.0                       0.0233   
1                               16.0                       0.0248   
2                               38.0                        0.059   
3                               12.0                       0.0186   
4                               13.0                       0.0202   

   Parte variable  Total final por socio  \
0             7.0                   17.0   
1             8.0                   18.0   
2 

In [8]:
# 7. Eliminar columna 'id' si existe
if 'id' in df_completo.columns:
    df_completo = df_completo.drop(columns=['id'])

In [9]:
def calcular_metricas_por_socio(df_socio):
    df_socio = df_socio.copy()
    
    # Stock del día anterior (desplazo la columna de pallets)
    df_socio['stock_ayer'] = df_socio['pallets'].shift(1)
    
    # Primer fila queda sin calcular, pero se usará como base para la siguiente
    df_socio['ingresos_estimados'] = df_socio['pallets'] - df_socio['stock_ayer'] + df_socio['cantidad_pallets_sacados']
    df_socio['cantidad_pallets_sacados_calc'] = df_socio['stock_ayer'] + df_socio['ingresos_estimados'] - df_socio['pallets']
    
    # Rotación diaria: pallets que salieron dividido stock del día anterior
    df_socio['rotacion_diaria'] = df_socio['cantidad_pallets_sacados'] / df_socio['stock_ayer']
    
    return df_socio

In [10]:
# 9. Aplicar por socio
df_completo = df_completo.groupby('id_socio', group_keys=False).apply(calcular_metricas_por_socio)

C:\Users\Sistemas\AppData\Local\Temp\ipykernel_8908\2045275685.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_completo = df_completo.groupby('id_socio', group_keys=False).apply(calcular_metricas_por_socio)


In [11]:
# 10. Borrar la primera fila de cada socio (la que no tiene datos útiles)
df_completo = df_completo[df_completo.groupby('id_socio').cumcount() != 0].reset_index(drop=True)

In [12]:
promedios = df_completo.groupby(['id_socio', 'mes'])['rotacion_diaria'].mean().reset_index()
promedios = promedios.rename(columns={'rotacion_diaria': 'promedio_rotacion_mensual'})
df_completo = pd.merge(df_completo, promedios, on=['id_socio', 'mes'], how='left')

In [13]:
ruta_destino = 'C:/Users/Sistemas/Desktop/pallets_cruzados_nuevo/cruce_2025/df_completo.xlsx'
df_completo.to_excel(ruta_destino)